# EXERCISE SOLUTION

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [2]:
# Constants

MODEL = "llama3:latest"

In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
AI Curriculum
Proficient AI Engineer
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
I will happily drone on for hours about LLMs to anyone in my vicinity. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To my total joy (and shock) they’ve become best-selling, top-rated courses, with 400,000 enrolled across 190 

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [5]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from Ollama expects the same message format as OpenAI:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [7]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - now with Ollama instead of OpenAI

In [8]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [9]:
summarize("https://edwarddonner.com")

"Here is a summary of the website, ignoring navigation-related text:\n\n**About Edward Donner**: Ed is the co-founder and CTO of Nebula.io, applying AI to help people discover their potential. He's also a Udemy instructor with best-selling courses on LLMs.\n\n**Posts**: The website features several blog posts, including articles on AI curriculum, proficient AI engineering, n8n, AI live events, MLOps track, and more.\n\nNo specific news or announcements were found on the website."

In [10]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
display_summary("https://edwarddonner.com")

Here is a summary of the website, ignoring navigation-related text:

**Welcome**
The website is about Edward Donner's work and projects related to Artificial Intelligence (AI) and Large Language Models (LLMs). He is a CTO at Nebula.io and has created best-selling Udemy courses on AI topics.

**Content**
The website contains information about his AI curriculum, including Proficient AI Engineer, Connect Four, and Outsmart. It also features blog posts discussing various AI-related topics, such as AI coder to Agentic Engineer, AI Builder with n8n, and more.

**About Edward Donner**
He is a passionate AI enthusiast who enjoys experimenting with LLMs and amateur electronic music production. He is grateful for his Udemy course students and invites visitors to stay in touch by subscribing to his newsletter or following him on social media platforms.

Overall, the website appears to be a personal space where Edward Donner shares his knowledge, experiences, and projects related to AI and LLMs.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [12]:
display_summary("https://cnn.com")

Here is a summary of the website in markdown:

**Summary**
The website is the official news website of CNN (Cable News Network), a 24-hour cable news channel that provides breaking news, videos, and podcasts on various topics including politics, business, entertainment, sports, and more.

**News Highlights**

* Towering civil rights leader Rev. Jesse Jackson dies at 84
* Peru's President José Jerí ousted, marking eighth change in leader in less than a decade
* Iran talks drag on, questions emerge over how long Trump will indulge diplomacy

**Videos**
The website features various video segments including news updates, analysis, and interviews.

**Sections**

* US: News, politics, business, entertainment, and more
* World: International news, politics, business, and more
* Business: Markets, pre-markets, after-hours, fear & greed, investing, and more
* Entertainment: Movies, television, celebrity, tech, innovate, and more

**Features**

* Analysis: In-depth analysis of current events and topics
* Video: Various video segments including news updates, interviews, and analysis
* Podcasts: Audio podcasts on various topics including politics, business, entertainment, and more
* Games: Daily crossword, jumble crossword, photo shuffle, sudoblock, sudoku, 5 things quiz, and more

In [13]:
display_summary("https://anthropic.com")

Here is a short summary of the website in markdown:

**About Anthropic**
Anthropic is a public benefit corporation dedicated to securing the benefits and mitigating the risks of AI. They build AI to serve humanity's long-term well-being.

**Products**
Anthropic offers several AI products, including:
- Claude: a space for thinking and conversation
- Claude Code: a model for coding, agents, and professional work
- Opus 4.6: a powerful model for coding, agents, and professional work
- Sonnet 4.6: the most capable Sonnet model yet with frontier performance

**AI Safety**
Anthropic emphasizes AI safety through:
- Responsible Scaling Policy
- Alignment Science
- Economic Index

**Announcements**
Recent announcements include:
- Claude is a space to think (February 4, 2026)
- Claude Opus 4.6 and Sonnet 4.6 releases (February 5 and 17, 2026)

**Company Information**
Anthropic provides information on their company, including:
- Careers
- Economic Futures
- Research
- News
- Transparency

Let me know if you'd like me to summarize anything else!

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c